# PROJECT 6. 멋진 작사가 만들기

## Step1. 데이터 수집
### (1) 데이터 다운로드
데이터는 노래 가사입니다. 아래의 명령어로 Song Lyrics 데이터를 다운로드 합니다.  

> ```$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip```  
```$ unzip song_lyrics.zip -d ~/폴더명```


## Step2. 데이터 점검 및 탐색
### (1) 데이터 불러오기
glob 모듈을 사용하여 파일을 불러옵니다.
> **glob.glob("폴더명/*.확장자명")** : 폴더에 있는 확장자명을 가진 파일만을 return 해준다.

In [1]:
import glob
import os
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

txt_file_path = os.getenv('HOME')+'/aiffel/exploration/lyricist/data/lyrics'    # 데이터가 있는 폴더경로
txt_list = glob.glob(txt_file_path + "/*.txt")    # txt 확장자를 가진 파일들을 다 가져오기


### (2) 데이터 살펴보기
불러온 txt 데이터를 한줄씩 읽어서 리스트에 저장합니다.

In [2]:
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()    # 파일안에 들어있는 모든 문장을 한줄 단위로 읽는다. (리스트로 저장됨)
        raw_corpus.extend(raw)    # Ex) 리스트 [1, 2, 3] -> 1, 2, 3 으로 리스트에 넣는다.

print(f"데이터 개수(텍스트 문장 개수) : {len(raw_corpus)}개")
print()

print("raw_corpus에 담긴 문장 10개 :")
for i in range(10):
    print(f"raw_corpus[{i}] : {raw_corpus[i]}")
print()

print(f"최대 문장 길이 : {len(max(raw_corpus, key=len))}")
print(f"최소 문장 길이 : {len(min(raw_corpus, key=len))}")
print()

print(f"빈 문장의 개수 : {len(raw_corpus) - len(list(filter(None, raw_corpus)))}")

데이터 개수(텍스트 문장 개수) : 187088개

raw_corpus에 담긴 문장 10개 :
raw_corpus[0] : 
raw_corpus[1] : 			“There must be some way out of here,” said the joker to the thief
raw_corpus[2] : “There’s too much confusion, I can’t get no relief
raw_corpus[3] : Businessmen, they drink my wine, plowmen dig my earth
raw_corpus[4] : None of them along the line know what any of it is worth”
raw_corpus[5] : 
raw_corpus[6] : “No reason to get excited,” the thief, he kindly spoke
raw_corpus[7] : “There are many here among us who feel that life is but a joke
raw_corpus[8] : But you and I, we’ve been through that, and this is not our fate
raw_corpus[9] : So let us not talk falsely now, the hour is getting late”

최대 문장 길이 : 1465
최소 문장 길이 : 0

빈 문장의 개수 : 11102


## Step3. 데이터 전처리
### (1) 데이터 정제 
- 모든 문장의 양쪽 공백을 삭제하고, 모두 소문자로 바꿉니다.  
> **```sentence.lower().strip()```**
- 문장에서 쓰이는 특수문자(?.!,¿)의 양쪽에 공백을 추가합니다.  
> **```re.sub(r"([?.!,¿])", r" \1 ", sentence)```**  
- 공백이 2개이상 연속되어 나오는 경우, 공백 1개로 바꿉니다.  
> **```re.sub(r'[" "]+', " ", sentence)```**  
- 문장에서 쓰이는 특수문자(?.!,¿)와 알파벳을 제외한 모든 문자를 공백으로 바꿉니다.  
> **```re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)```**  

In [3]:
# 데이터 정제
import re 

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()    # 양쪽 공백 삭제하기

    sentence = '<start> ' + sentence + ' <end>'      # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

corpus = []

for sentence in raw_corpus:
    if (len(sentence) == 0) or (len(preprocess_sentence(sentence).split()) >= 16): # 토큰의 개수가 15개가 넘어가는 문장 제외
        continue

    corpus.append(preprocess_sentence(sentence))
        
print(f"전처리한 데이터 개수(텍스트 문장 개수) : {len(corpus)}개")
print()

print("corpus에 담긴 문장 10개 :")
for i in range(10):
    print(f"corpus[{i}] : {corpus[i]}")


전처리한 데이터 개수(텍스트 문장 개수) : 156227개

corpus에 담긴 문장 10개 :
corpus[0] : <start> there s too much confusion , i can t get no relief <end>
corpus[1] : <start> businessmen , they drink my wine , plowmen dig my earth <end>
corpus[2] : <start> none of them along the line know what any of it is worth <end>
corpus[3] : <start> no reason to get excited , the thief , he kindly spoke <end>
corpus[4] : <start> so let us not talk falsely now , the hour is getting late <end>
corpus[5] : <start> all along the watchtower , princes kept the view <end>
corpus[6] : <start> while all the women came and went , barefoot servants , too <end>
corpus[7] : <start> outside in the distance a wildcat did growl <end>
corpus[8] : <start> two riders were approaching , the wind began to howl <end>
corpus[9] : <start> once upon a time you dressed so fine <end>


### (2) 단어 사전 구축하기
- 문자열을 숫자로 변환해주는 작업입니다. 문장을 공백단위로 토큰화해서 나온 모든 단어들을 사전으로 만듭니다.  
    ex) I like you and I love you → {0 : 'I', 1: 'you', 2: 'like',  3: 'and', 4: 'love'}*  
> **tf.keras.preprocessing.text.Tokenizer(num_words, filters, ovv_token)**  

- 문장 맨 끝에 패딩을 함으로써 문장의 길이를 모두 똑같게 맞춥니다.  
> **tf.keras.preprocessing.sequence.pad_sequences(tensor, padding, maxlen)** 


In [4]:
# 사전 구축
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.
    print(f"문장에 쓰인 총 단어수 : {len(tokenizer.word_index)}")
    print(f"가장 긴 문장의 길이 : {len(max(corpus, key=len).split())}")
    print()
    
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.   
    print("1. 구축한 사전으로부터 corpus를 해석해 tensor로 변환")
    print(f"corpus[0] : {corpus[0]}")
    print(f"tensor[0] : {tensor[0]} → 길이 : {len(tensor[0])}")
    print()
    print(f"corpus[1] : {corpus[1]}")
    print(f"tensor[1] : {tensor[1]} → 길이 : {len(tensor[1])}")
    print()
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    print(f"2. 길이 15 로 맞추기")
    print("tensor[0]")
    print(f": {tensor[0]}")
    print()
    print("tensor[1]")
    print(f": {tensor[1]}")
    print()

    return tensor, tokenizer

# tensor는 데이터, tokenizer는 사전
# 텐서 데이터는 모두 정수로 이루어져 있으며, 이 숫자는 tokenizer에 구축된 사전의 인덱스입니다.
tensor, tokenizer = tokenize(corpus)

# 구축된 사전 확인하기
print("구축된 사전 10개")
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break
        
        

문장에 쓰인 총 단어수 : 25692
가장 긴 문장의 길이 : 15

1. 구축한 사전으로부터 corpus를 해석해 tensor로 변환
corpus[0] : <start> there s too much confusion , i can t get no relief <end>
tensor[0] : [2, 62, 17, 102, 184, 2718, 5, 4, 35, 16, 43, 41, 5747, 3] → 길이 : 14

corpus[1] : <start> businessmen , they drink my wine , plowmen dig my earth <end>
tensor[1] : [2, 17664, 5, 38, 472, 13, 924, 5, 17665, 1066, 13, 646, 3] → 길이 : 13

2. 길이 15 로 맞추기
tensor[0]
: [   2   62   17  102  184 2718    5    4   35   16   43   41 5747    3
    0]

tensor[1]
: [    2 17664     5    38   472    13   924     5 17665  1066    13   646
     3     0     0]

구축된 사전 10개
1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


### (3) 소스 문장과 타겟 문장 생성하기
- 우리의 목적은 어떤 단어를 입력했을때, 그 단어 뒤에 무슨 단어가 올지 출력하는것입니다.  
    이를 위해서 입력 데이터와 출력 데이터를 따로 생성해야합니다.
> total sentence : "I love my music."  
source sentence : "I love my"
target sentence : "love my music."

     

In [5]:
# 텐서 데이터를 입력 데이터와 출력 데이터로 나누기
# 입력 데이터 : 한 문장안에서의 마지막 단어를 제외한 모든 단어, 출력 데이터 : 한문장 안에서 첫번째 단어를 제외한 모든 단어
# 사전에 <pad> 를 추가하지 않아도 된다. 텐서가 0을 알아서 패딩문자로 인식한다.

src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(f"0번째 전체 데이터 : {tensor[0]}")
print(f"0번째 소스 데이터 : {src_input[0]}")
print(f"0번째 타겟 데이터 : {tgt_input[0]}")


0번째 전체 데이터 : [   2   62   17  102  184 2718    5    4   35   16   43   41 5747    3
    0]
0번째 소스 데이터 : [   2   62   17  102  184 2718    5    4   35   16   43   41 5747    3]
0번째 타겟 데이터 : [  62   17  102  184 2718    5    4   35   16   43   41 5747    3    0]


### (3) 배치 데이터 생성하기
지금까지 전처리한 데이터들을 학습에 맞는 데이터로 변환시켜야합니다.
> **tf.data.Dataset.from_tensor_slices().batch(BATCH_SIZE, drop_remainder=True)**  

In [6]:
# 데이터 셋 객체 생성
# model.fit 은 numpy array 데이터 셋이 필요
# tensorflow는 tensor 데이터 셋 필요 : tf.data.Dataset.from_tensor_slices() 으로 tensor 데이터 셋 생성가능

BUFFER_SIZE = len(src_input)    # 총 데이터의 갯수 : 175986 개
BATCH_SIZE = 128
STEPS_PER_EPOCH = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

# print("* batch하기 전 : 인공지능이 한번 학습할 양")
# for i in dataset:
#     print(i)
#     break
#     
# print()
# 
# batch size : 한번에 학습되는 데이터 개수
# 하나의 데이터를 여러개로 나누어 학습한다.
# dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# print("* batch 후 : 인공지능이 한번 학습할 양")
# for i in dataset:
#     print(i)
#     break
# print()
    

### (4) 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리합니다.
> **train_test_split()**  

In [7]:
# 훈련 데이터와 평가 데이터 분리하기
from sklearn.model_selection import train_test_split

# numpy_dataset = dataset.numpy()
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print()

print("Source Validation:", enc_val.shape)
print("Target Validation:", dec_val.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)

Source Validation: (31246, 14)
Target Validation: (31246, 14)


## Step4. 모델링 및 훈련
### (1) 모델 구현하기
- 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성된 모델을 만듭니다.

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model.build(enc_train.shape)
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  10240512  
_________________________________________________________________
lstm_2 (LSTM)                multiple                  6295552   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  20501025  
Total params: 45,429,793
Trainable params: 45,429,793
Non-trainable params: 0
_________________________________________________________________


### (2) 학습시키기


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

model.fit(enc_train, 
          dec_train, 
          epochs=5, 
          batch_size=BATCH_SIZE,
          validation_data=(enc_val, dec_val),
          steps_per_epoch=STEPS_PER_EPOCH,
          verbose=1)

Epoch 1/5
1220/1220 [==============================] - 166s 136ms/step - loss: 2.4593 - val_loss: 2.7085
Epoch 2/5
1220/1220 [==============================] - 169s 139ms/step - loss: 2.2606 - val_loss: 2.6675
Epoch 3/5
1220/1220 [==============================] - 168s 137ms/step - loss: 2.1162 - val_loss: 2.6396
Epoch 4/5
1220/1220 [==============================] - 168s 138ms/step - loss: 1.9965 - val_loss: 2.6250
Epoch 5/5
   5/1220 [..............................] - 8s 2s/step - loss: 1.9813 - val_loss: 2.6220


## Step5. 평가
모델이 작문을 잘하는지 평가합니다. 작문 모델을 평가하는 방법은 작문을 시켜보고 직접 평가하는 것입니다.  
아래 코드의 generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행합니다.

In [26]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

answer1 = generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)
answer2 = generate_text(model, tokenizer, init_sentence="<start> i loved", max_len=15)
answer3 = generate_text(model, tokenizer, init_sentence="<start> love", max_len=15)

print(f"i love → {answer1}")
print(f"i loved → {answer2}")
print(f"love → {answer3}")

i love → <start> i love you , i m a bad girl <end> 
i loved → <start> i loved you <end> 
love → <start> love is a losing game <end> 


## [ 결과 - 루브릭 ]

#### 1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
- 텍스트 제네레이션 결과가 그럴듯한 문장으로 생성되었습니다. :)  

#### 2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
- 특수문자 제거, 토크나이저 생성, 패딩 처리의 과정을 진행하였습니다 :)

#### 3. 텍스트 생성모델이 안정적으로 학습되었는가?
- 텍스트 생성모델의 validation loss는 2.6 으로, 2.2 이하의 값이 나오지 못했습니다. :(

## [느낀점]
이번 프로젝트는 다른 프로젝트들보다 소스코드를 짜는데 많이 어렵지는 않았습니다.  
하지만 validation loss를 줄이는게 상당히 힘들었습니다.  
loss를 줄이기 위해서 hidden_size, batch_size, embedding_size, epoch 를 변경해봤는데, loss에 큰 변화가 없었습니다.  
그나마 batch_size를 변경하니까 loss가 조금 낮아졌습니다.  
제가 아직 딥러닝에 사용되는 값들에 대해 잘 알지 못해서 좋은 모델을 구현하지 못한 것 같습니다.  
앞으로의 프로젝트를 위해서 딥러닝에 대해서 더 공부해야 겠다는 필요성을 느끼게 되었습니다.  
    
> **loss 값을 줄이기 위해서는 모델을 어떻게 구현해야할까?**

<br/>

loss 값을 줄이려고 hidden_size, batch_size, embedding_size, epoch 다 바꿔봤는데도 안 줄어드네요....  
팀원분들은 잘 마무리 하신것 같은데... 제가 많이 부족한가 봅니다...  
**앞으로 더 열심히 해야겠습니다 ㅠ.ㅠ**
